# Flask REST Part 1

## Getting Started

First, you’ll create a simple web server using the Flask Micro Framework. To get started, create a directory where you can create the code. You should also work in a virtualenv so you can install modules later on, which you’ll need to do. In addition, create a templates directory.

The Python code below gets a very basic web server up and running, and responding with Hello World for a request for the home page:

In [ ]:
from flask import (
    Flask,
    render_template
)

# Create the application instance
app = Flask(__name__, template_folder="templates")

# Create a URL route in our application for "/"
@app.route('/')
def home():
    """
    This function just responds to the browser ULR
    localhost:5000/

    :return:        the rendered template 'home.html'
    """
    return render_template('home.html')

# If we're running in stand alone mode, run the application
if __name__ == '__main__':
    app.run(debug=True)

You should also create a home.html in the templates folder, as this is what will be served to a browser when navigating to the URL '/'. Here is the contents of the home.html file:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Application Home Page</title>
</head>
<body>
    <h2>
        Hello World!
    </h2>
</body>
</html>
```

You’ll notice the HTML file is named home.html rather than index.html. This is intentional because having an index.html file in the templates directory causes problems once you import the Connexion module in your program.

You can run your application with this command line in the directory containing the server.py file with the Python VirtualEnv active:

`python server.py`

When you run this application, a web server will start on port 5000, which is the default port used by Flask. If you open a browser and navigate to localhost:5000, you should see Hello World! displayed. Right now, this is useful to see the web server is running. You’ll extend the home.html file later to become a full single page web application using the REST API you’re developing.

In your Python program, you’ve imported the Flask module, giving the application access to the Flask functionality. You then created a Flask application instance, the app variable. Next, you connected the URL route '/' to the home() function by decorating it with @app.route('/'). This function calls the Flask render_template() function to get the home.html file from the templates directory and return it to the browser.

## Using Connexion to Add a REST API Endpoint

Now that you’ve got a working web server, let’s add a REST API endpoint. To do this, you’ll use the Connexion module, which is installed using pip:

`$ pip install connexion[swagger-ui]`

This makes the Connexion module available to your program. The Connexion module allows a Python program to use the Swagger specification. This provides a lot of functionality: validation of input and output data to and from your API, an easy way to configure the API URL endpoints and the parameters expected, and a really nice UI interface to work with the created API and explore it.

All of this can happen when you create a configuration file your application can access. The Swagger site even provides an online configuration editor tool to help create and/or syntax check the configuration file you will create.

## Adding Connexion to the Server

There are two parts to adding a REST API URL endpoint to your application with Connexion. You’ll add Connexion to the server and create a configuration file it will use. Modify your Python program like this to add Connexion to the server:

```py
from flask import render_template
import connexion

# Create the application instance
app = connexion.App(__name__, specification_dir='./')

# Read the swagger.yml file to configure the endpoints
app.add_api('swagger.yml')

# Create a URL route in our application for "/"
@app.route('/')
def home():
    """
    This function just responds to the browser ULR
    localhost:5000/
    :return:        the rendered template 'home.html'
    """
    return render_template('home.html')

# If we're running in stand alone mode, run the application
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
```

You’ve added a couple of things to incorporate Connexion into the server. The import connexion statement adds the module to the program. The next step is creating the application instance using Connexion rather than Flask. Internally, the Flask app is still created, but it now has additional functionality added to it.

Part of the app instance creation includes the parameter specification_dir. This informs Connexion what directory to look in for its configuration file, in this case our current directory. Right after this, you’ve added the line:

`app.add_api('swagger.yml')`

This tells the app instance to read the file swagger.yml from the specification directory and configure the system to provide the Connexion functionality.

## The Swagger Configuration File

The file swagger.yml is a YAML or JSON file containing all of the information necessary to configure your server to provide input parameter validation, output response data validation, URL endpoint definition, and the Swagger UI. Here is the swagger.yml file defining the GET `/api/people` endpoint your REST API will provide:

```yaml
swagger: "2.0"
info:
  description: This is the swagger file that goes with our server code
  version: "1.0.0"
  title: Swagger REST Article
consumes:
  - "application/json"
produces:
  - "application/json"

basePath: "/api"

# Paths supported by the server application
paths:
  /people:
    get:
      operationId: "people.read"
      tags:
        - "People"
      summary: "The people data structure supported by the server application"
      description: "Read the list of people"
      responses:
        200:
          description: "Successful read people list operation"
          schema:
            type: "array"
            items:
              properties:
                fname:
                  type: "string"
                lname:
                  type: "string"
                timestamp:
                  type: "string"
```

This file is organized in a hierarchical manner: the indentation levels represent a level of ownership, or scope.

For example, paths defines the beginning of where all the API URL endpoints are defined. The /people value indented under that defines the start of where all the `/api/people` URL endpoints will be defined. The get: indented under that defines the section of definitions associated with an HTTP GET request to the `/api/people` URL endpoint. This goes on for the entire configuration.

Here are the meanings of the fields in this section of the swagger.yml file:

This section is part of the global configuration information:

1. swagger: tells Connexion what version of the Swagger API is being used
2. info: begins a new ‘scope’ of information about the API being built
3. description: a user defined description of what the API provides or is about. This is in the Connexion generated UI system
4. version: a user defined version value for the API
5. title: a user defined title included in the Connexion generated UI system
6. consumes: tells Connexion what MIME type is expected by the API.
7. produces: tells Connexion what content type is expected by the caller of the API.
8. basePath: "/api" defines the root of the API, kind of like a namespace the REST API will come from.

This section begins the configuration of the API URL endpoint paths:

1. paths: defines the section of the configuration containing all of the API REST endpoints.
2. /people: defines one path of your URL endpoint.
3. get: defines the HTTP method this URL endpoint will respond to. Together with the previous definitions, this creates the GET `/api/people` URL endpoint.

This section begins the configuration of the single `/api/people` URL endpoint:

1. operationId: "people.read" defines the Python import path/function that will respond to an HTTP GET `/api/people` request. The "people.read" portion can go as deep as you need to in order to connect a function to the HTTP request. Something like `"<package_name>.<package_name>.<package_name>.<function_name>"` would work just as well. You’ll create this shortly.
2. tags: defines a grouping for the UI interface. All the CRUD methods you’ll define for the people endpoint will share this tag definition.
summary defines the UI interface display text for this endpoint.
3. description: defines what the UI interface will display for implementation notes.

This section defines the section of the configuration of a successful response from the URL endpoint:

1. response: defines the beginning of the expected response section.
1. 200: defines the section for a successful response, HTTP status code 200.
1. description: defines the UI interface display text for a response of 200.
1. schema: defines the response as a schema, or structure.
1. type: defines the structure of the schema as an array.
1. items: starts the definition of the items in the array.
1. properties: defines the items in the array as objects having key/value pairs.
1. fname: defines the first key of the object.
1. type: defines the value associated with fname as a string.
1. lname: defines the second key of the object.
1. type: defines the value associated with lname as a string.
1. timestamp: defines the third key of the object.
1. type: defines the value associated with timestamp as a string.

## Handler for People Endpoint

In the swagger.yml file, you configured Connexion with the operationId value to call the people module and the read function within the module when the API gets an HTTP request for GET `/api/people`. This means a people.py module must exist and contain a read() function. Here is the people.py module you will create:

```py
from datetime import datetime

def get_timestamp():
    return datetime.now().strftime(("%Y-%m-%d %H:%M:%S"))

# Data to serve with our API
PEOPLE = {
    "Farrell": {
        "fname": "Doug",
        "lname": "Farrell",
        "timestamp": get_timestamp()
    },
    "Brockman": {
        "fname": "Kent",
        "lname": "Brockman",
        "timestamp": get_timestamp()
    },
    "Easter": {
        "fname": "Bunny",
        "lname": "Easter",
        "timestamp": get_timestamp()
    }
}

# Create a handler for our read (GET) people
def read():
    """
    This function responds to a request for /api/people
    with the complete lists of people

    :return:        sorted list of people
    """
    # Create the list of people from our data
    return [PEOPLE[key] for key in sorted(PEOPLE.keys())]
```

In this code, you’ve created a helper function called get_timestamp() that generates a string representation of the current timestamp. This is used to create your in-memory structure and modify the data when you start modifying it with the API.

You then created the PEOPLE dictionary data structure, which is a simple names database, keyed on the last name. This is a module variable, so its state persists between REST API calls. In a real application, the PEOPLE data would exist in a database, file, or network resource, something that persists the data beyond running/stopping the web application.

Then you created the read() function. This is called when an HTTP request to GET `/api/people` is received by the server. The return value of this function is converted to a JSON string (recall the produces: definition in the swagger.yml file). The read() function you created builds and returns a list of people sorted by last name.

Running your server code and navigating in a browser to localhost:5000/api/people will display the list of people on screen:

```js
[
  {
    "fname": "Kent",
    "lname": "Brockman",
    "timestamp": "2018-05-10 18:12:42"
  },
  {
    "fname": "Bunny",
    "lname": "Easter",
    "timestamp": "2018-05-10 18:12:42"
  },
  {
    "fname": "Doug",
    "lname": "Farrell",
    "timestamp": "2018-05-10 18:12:42"
  }
]
```

Congratulations, you’ve created a nice API and are on your way to building out a complete one!

## The Swagger UI

Now you have a simple web API running with a single URL endpoint. At this point, it would be reasonable to think, “configuring this with the swagger.yml file was cool and all, but what did it get me?”

You’d be right to think that. We haven’t taken advantage of the input or output validation. All that swagger.yml gave us was a definition for the code path connected to the URL endpoint. However, what you also get for the extra work is the creation of the Swagger UI for your API.

If you navigate to `localhost:5000/api/ui`, the system will bring up a page.

This is the initial Swagger interface and shows the list of URL endpoints supported at our `localhost:5000/api` endpoint. This is built automatically by Connexion when it parses the swagger.yml file.

If you click on the `/people` endpoint in the interface, the interface will expand to show a great deal more information about your API.

This displays the structure of the expected response, the content-type of that response, and the description text you entered about the endpoint in the swagger.yml file.

You can even try the endpoint out by clicking the `Try It Out!` button at the bottom of the screen.

## Building Out the Complete API

Our original goal was to build out an API providing full CRUD access to our people structure.

To achieve this, you’ll extend both the swagger.yml and people.py files to fully support the API defined above. For the sake of brevity, only a link will be provided for both files:

- [swagger.yml](https://drive.google.com/file/d/1cgX_pJPH0AoF95GVRdndpIiB4riJEcHR/view?usp=sharing)
- [people.py](https://drive.google.com/file/d/1kIhZc_ZN48Ny6etG7YwUxHbIwP89cPPg/view?usp=sharing)




## Demonstration Single-Page Application

You’ve got a working REST API with a great Swagger UI documentation/interaction system. But what do you do with it now? The next step is to create a web application demonstrating the use of the API in a semi-practical manner.

You’ll create a web application that displays the people on screen as well as allows the user to create new people, update existing people, and delete people. This will all be handled by AJAX calls from JavaScript to the people API URL endpoints.

To begin, you need to extend the home.html file to look like this:

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Application Home Page</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.0/normalize.min.css">
    <link rel="stylesheet" href="static/css/home.css">
    <script
      src="http://code.jquery.com/jquery-3.3.1.min.js"
      integrity="sha256-FgpCb/KJQlLNfOu91ta32o/NMZxltwRo8QtmkMRdAu8="
      crossorigin="anonymous">
    </script>
</head>
<body>
    <div class="container">
        <h1 class="banner">People Demo Application</h1>
        <div class="section editor">
            <label for="fname">First Name
                <input id="fname" type="text" />
            </label>
            <br />
            <label for="lname">Last Name
                <input id="lname" type="text" />
            </label>
            <br />
            <button id="create">Create</button>
            <button id="update">Update</button>
            <button id="delete">Delete</button>
            <button id="reset">Reset</button>
        </div>
        <div class="people">
            <table>
                <caption>People</caption>
                <thead>
                    <tr>
                        <th>First Name</th>
                        <th>Last Name</th>
                        <th>Update Time</th>
                    </tr>
                </thead>
                <tbody>
                </tbody>
            </table>
        </div>
        <div class="error">
        </div>
    </div>
</body>
<script src="static/js/home.js"></script>
</html>
```

The above HTML code extends the home.html file to pull in the external normalize.min.css file, which is a CSS reset file to normalize the formatting of elements across browsers.

It also pulls in the external jquery-3.3.1.min.js file to provide the jQuery functionality you’ll use to create the single-page application interactivity.

The HTML code above creates the static framework of the application. The dynamic parts appearing in the table structure will be added by JavaScript at load time and as the user interacts with the application.

## Static Files

In the home.html file you’ve created, there are references to two static files: static/css/home.css and static/js/home.js. To add these, you’ll need to add the following directory structure to the application:

```dir
static/
│
├── css/
│   └── home.css
│
└── js/
    └── home.js
```

Because a directory named static will automatically be served by the Flask application, any files you place in the css and js folders will be available to the home.html file. For the sake of brevity, here are links to the home.css and home.js files:

- [home.css](https://drive.google.com/file/d/1l50AlEMIBocxP0RalVpHYmufcioRuakR/view?usp=sharing)
- [home.js](https://drive.google.com/file/d/1MyD3EbEwuwUA3Tr2oIhNefMKcY-CBNfe/view?usp=sharing)

## JavaScript File

As was mentioned, the JavaScript file provides all the interaction with and updates to the web application. It does this by breaking up the necessary functionality into three parts by using the MVC (Model / View / Controller) design pattern.

Each object is created by a self-invoking function returning its own API for use by the other pieces. For instance, the Controller is passed the instances of the Model and View as parameters in its instantiation. It interacts with those objects through their exposed API methods.

The only other connection is between the Model and Controller by means of custom events on the AJAX method calls:

1. The Model provides the connection to the people API. Its own API is what the Controller calls to interact with the server when a user interaction event requires it.
2. The View provides the mechanism to update the web application DOM. Its own API is what the Controller calls to update the DOM when a user interaction event requires it.
3. The Controller provides all the event handlers for user interaction with the web application. This allows it to make calls to the Model to make requests to the people API, and to the View to update the DOM with new information received from the people API.

It also handles the custom events generated by the asynchronous AJAX requests made by the Model to the people API.

Complete file can be downloaded [here](https://github.com/ardhiraka/H8Flask/tree/master/version1).